# Project: Wrangle and Analyse Twitter's WeRateDogs 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling Process</a></li>
    <ul>
        <li><a href="#gathering">Gathering Data</a></li>
        <li><a href="#assessing">Assessing Data</a></li>
        <li><a href="#cleaning">Cleaning Data</a></li>
    </ul>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id="intro"></a>
## Introduction

In [83]:
# Importing the main packages for this project

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

<a id="wrangling"></a>
## Data Wrangling Process

The data wrangling process consist of three steps: 
- **Gathering Data** 
- **Assessing Data**
- **Cleaning Data**.

In [84]:
# Importing the main packages for the data wrangling process

import requests
import tweepy
import json
import time

<a id="gathering"></a>
### Gathering Data

The data for this project comes from three different sources using three different methods. 

1. **WeRateDogs Twitter Archive**, obtained by **downloading it manually** on [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)

2. **Tweet Image Predictions**, i.e., what breed of dog is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and will be **downloaded programmatically** using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv.

3. **Tweet Information**, including each tweet's retweet count, favourite ("like") count, and any additional interesting data. This is obtained by **querying Twitter's API**. 

#### 1. WeRateDogs archive

File `twitter-archive-enhanced.csv` is already downloaded manually. 

In [240]:
# create DF for WeRateDogs Twitter archive (file already downloaded manually)
df_dogs_archive = pd.read_csv("twitter-archive-enhanced.csv")
df_dogs_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,nan,nan,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,nan,nan,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",nan,nan,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,nan,nan,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,nan,nan,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,nan,nan,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",nan,nan,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


#### 2. Tweet Image Prediction from Udacity server

The file `image-predictions.tsv` is hosted on the Udacity server and can be downloaded programmatically using the `requests` library, then saved to file, and then DataFrame is created. 

In [4]:
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

with open("image-predictions.tsv", mode="wb") as file:
    file.write(response.content)

In [73]:
# create DF for tweet image prediction
df_image_pred = pd.read_csv("image-predictions.tsv", sep="\t")
df_image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


#### 3. Tweet information by querying Twitter API

A developer account is created and approved at Twitter. Then an app is created, where the keys and tokens can be obtained.

Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library. This is then stored in a file. Interesting data is extracted and this is then read into a pandas DataFrame. 

***Note: the code below shows how to obtain data by querying Twitter API. The actual file used is "tweet_json.txt" provided by Udacity. This will be used for the analysis for the remainder of the project (and not the one queried from Twitter). This allows for consistency and reproducibility.***

In [6]:
# Consumer and access keys, tokens, and secrets from Twitter Developer App
# These are hidden to comply with Twitter's API terms and conditions
consumer_key = "" # Hidden
consumer_secret = ""
access_token = ""
access_secret = ""

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# Twitter's has a 15 minute rate limit, and the query will take longer
# so API will continue 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [8]:
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
# Each tweet's JSON data is written to its own line. 

count = 0
fails = {}
start = time.time()

with open("tweet_json_api_created.txt", "w") as outfile:
    for tweet in we_rate_dogs.tweet_id:
        count += 1
        try:
            tweet_api = api.get_status(tweet, tweet_mode='extended')
            #tweet_api returns a Status object. To access the JSON file, use ._json
            json.dump(tweet_api._json, outfile) 
            outfile.write("\n")
            print("Success - count {}: {}".format(count, tweet))
        # Skip deleted tweets
        except tweepy.TweepError as e:
            fails[tweet] = e
            print("Fail - count {}: {}".format(count, tweet))
            pass

end = time.time()
print(end - start)

Success - count 1: 892420643555336193
Success - count 2: 892177421306343426
Success - count 3: 891815181378084864
Success - count 4: 891689557279858688
Success - count 5: 891327558926688256
Success - count 6: 891087950875897856
Success - count 7: 890971913173991426
Success - count 8: 890729181411237888
Success - count 9: 890609185150312448
Success - count 10: 890240255349198849
Success - count 11: 890006608113172480
Success - count 12: 889880896479866881
Success - count 13: 889665388333682689
Success - count 14: 889638837579907072
Success - count 15: 889531135344209921
Success - count 16: 889278841981685760
Success - count 17: 888917238123831296
Success - count 18: 888804989199671297
Success - count 19: 888554962724278272
Fail - count 20: 888202515573088257
Success - count 21: 888078434458587136
Success - count 22: 887705289381826560
Success - count 23: 887517139158093824
Success - count 24: 887473957103951883
Success - count 25: 887343217045368832
Success - count 26: 88710139280408576

Rate limit reached. Sleeping for: 248


Success - count 111: 871102520638267392
Success - count 112: 871032628920680449
Success - count 113: 870804317367881728
Success - count 114: 870726314365509632
Success - count 115: 870656317836468226
Success - count 116: 870374049280663552
Success - count 117: 870308999962521604
Success - count 118: 870063196459192321
Fail - count 119: 869988702071779329
Success - count 120: 869772420881756160
Success - count 121: 869702957897576449
Success - count 122: 869596645499047938
Success - count 123: 869227993411051520
Success - count 124: 868880397819494401
Success - count 125: 868639477480148993
Success - count 126: 868622495443632128
Success - count 127: 868552278524837888
Success - count 128: 867900495410671616
Success - count 129: 867774946302451713
Success - count 130: 867421006826221569
Success - count 131: 867072653475098625
Success - count 132: 867051520902168576
Fail - count 133: 866816280283807744
Success - count 134: 866720684873056260
Success - count 135: 866686824827068416
Succes

Success - count 317: 834931633769889797
Success - count 318: 834786237630337024
Success - count 319: 834574053763584002
Success - count 320: 834477809192075265
Success - count 321: 834458053273591808
Success - count 322: 834209720923721728
Success - count 323: 834167344700198914
Success - count 324: 834089966724603904
Success - count 325: 834086379323871233
Success - count 326: 833863086058651648
Success - count 327: 833826103416520705
Success - count 328: 833732339549220864
Success - count 329: 833722901757046785
Success - count 330: 833479644947025920
Success - count 331: 833124694597443584
Success - count 332: 832998151111966721
Success - count 333: 832769181346996225
Success - count 334: 832757312314028032
Success - count 335: 832682457690300417
Success - count 336: 832645525019123713
Success - count 337: 832636094638288896
Success - count 338: 832397543355072512
Success - count 339: 832369877331693569
Success - count 340: 832273440279240704
Success - count 341: 832215909146226688


Success - count 523: 809808892968534016
Success - count 524: 809448704142938112
Success - count 525: 809220051211603969
Success - count 526: 809084759137812480
Success - count 527: 808838249661788160
Success - count 528: 808733504066486276
Success - count 529: 808501579447930884
Success - count 530: 808344865868283904
Success - count 531: 808134635716833280
Success - count 532: 808106460588765185
Success - count 533: 808001312164028416
Success - count 534: 807621403335917568
Success - count 535: 807106840509214720
Success - count 536: 807059379405148160
Success - count 537: 807010152071229440
Success - count 538: 806629075125202948
Success - count 539: 806620845233815552
Success - count 540: 806576416489959424
Success - count 541: 806542213899489280
Success - count 542: 806242860592926720
Success - count 543: 806219024703037440
Success - count 544: 805958939288408065
Success - count 545: 805932879469572096
Success - count 546: 805826884734976000
Success - count 547: 805823200554876929


...

Success - count 1811: 676821958043033607
Success - count 1812: 676819651066732545
Success - count 1813: 676811746707918848
Success - count 1814: 676776431406465024
Success - count 1815: 676617503762681856
Success - count 1816: 676613908052996102
Success - count 1817: 676606785097199616
Success - count 1818: 676603393314578432
Success - count 1819: 676593408224403456
Success - count 1820: 676590572941893632
Success - count 1821: 676588346097852417
Success - count 1822: 676582956622721024
Success - count 1823: 676575501977128964
Success - count 1824: 676533798876651520
Success - count 1825: 676496375194980353
Success - count 1826: 676470639084101634
Success - count 1827: 676440007570247681
Success - count 1828: 676430933382295552
Success - count 1829: 676263575653122048
Success - count 1830: 676237365392908289
Success - count 1831: 676219687039057920
Success - count 1832: 676215927814406144
Success - count 1833: 676191832485810177
Success - count 1834: 676146341966438401
Success - count 

Rate limit reached. Sleeping for: 262


Success - count 1911: 674416750885273600
Success - count 1912: 674410619106390016
Success - count 1913: 674394782723014656
Success - count 1914: 674372068062928900
Success - count 1915: 674330906434379776
Success - count 1916: 674318007229923329
Success - count 1917: 674307341513269249
Success - count 1918: 674291837063053312
Success - count 1919: 674271431610523648
Success - count 1920: 674269164442398721
Success - count 1921: 674265582246694913
Success - count 1922: 674262580978937856
Success - count 1923: 674255168825880576
Success - count 1924: 674082852460433408
Success - count 1925: 674075285688614912
Success - count 1926: 674063288070742018
Success - count 1927: 674053186244734976
Success - count 1928: 674051556661161984
Success - count 1929: 674045139690631169
Success - count 1930: 674042553264685056
Success - count 1931: 674038233588723717
Success - count 1932: 674036086168010753
Success - count 1933: 674024893172875264
Success - count 1934: 674019345211760640
Success - count 

Success - count 2111: 670444955656130560
Success - count 2112: 670442337873600512
Success - count 2113: 670435821946826752
Success - count 2114: 670434127938719744
Success - count 2115: 670433248821026816
Success - count 2116: 670428280563085312
Success - count 2117: 670427002554466305
Success - count 2118: 670421925039075328
Success - count 2119: 670420569653809152
Success - count 2120: 670417414769758208
Success - count 2121: 670411370698022913
Success - count 2122: 670408998013820928
Success - count 2123: 670403879788544000
Success - count 2124: 670385711116361728
Success - count 2125: 670374371102445568
Success - count 2126: 670361874861563904
Success - count 2127: 670338931251150849
Success - count 2128: 670319130621435904
Success - count 2129: 670303360680108032
Success - count 2130: 670290420111441920
Success - count 2131: 670093938074779648
Success - count 2132: 670086499208155136
Success - count 2133: 670079681849372674
Success - count 2134: 670073503555706880
Success - count 

Success - count 2311: 666786068205871104
Success - count 2312: 666781792255496192
Success - count 2313: 666776908487630848
Success - count 2314: 666739327293083650
Success - count 2315: 666701168228331520
Success - count 2316: 666691418707132416
Success - count 2317: 666649482315059201
Success - count 2318: 666644823164719104
Success - count 2319: 666454714377183233
Success - count 2320: 666447344410484738
Success - count 2321: 666437273139982337
Success - count 2322: 666435652385423360
Success - count 2323: 666430724426358785
Success - count 2324: 666428276349472768
Success - count 2325: 666421158376562688
Success - count 2326: 666418789513326592
Success - count 2327: 666411507551481857
Success - count 2328: 666407126856765440
Success - count 2329: 666396247373291520
Success - count 2330: 666373753744588802
Success - count 2331: 666362758909284353
Success - count 2332: 666353288456101888
Success - count 2333: 666345417576210432
Success - count 2334: 666337882303524864
Success - count 

***Note: for consistency and reproducibility, the remainder of the project will use "tweet_json.txt" provided by Udacity, and not the one queried from Twitter API above.***

In [78]:
# tweet_json.txt file is read tweet by tweet to extract interesting information:
# tweet ID, retweet count, and favorite count. Add these into a list
df_list = []

with open("tweet_json.txt") as json_file:
    for line in json_file: 
        tweet = json.loads(line)
        tweet_id = tweet["id"]
        retweet_count = tweet["retweet_count"]
        favorite_count = tweet["favorite_count"]
        df_list.append({"tweet_id": tweet_id,
                        "retweet_count": retweet_count,
                        "favorite_count": favorite_count})

In [79]:
df_tweets = pd.DataFrame(df_list, columns=["tweet_id", "retweet_count", 
                                           "favorite_count"])
df_tweets.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


<a id="gathering"></a>
### Assessing Data

Assessing data reveals issues that can be categorised into two main types:
- **Quality issues**: due to the data's content. This is also known as dirty data or low quality data
- **Tidiness issues**: due to the data's structure. This is also known as untidy data

There are two ways to identify such issues: **visual** and **programmatic assessments**. 

The following data quality dimensions will help guide through assessing and cleaning data:

- **Completeness** - is the data complete?
- **Validity** - are all records valid and confrom to a defined schema?
- **Accuracy** - is all the data accurate even if they are valid?
- **Consistency** - do similar data within and across tables have a consistent format?

The data quality and tidiness issues will be summarised at the end of this section. 

***Dataset 1: WeRateDogs archive `df_dogs_archive`***

Column descriptions:
- `tweet_id`: last part of the tweet URL after "status/", e.g. https://twitter.com/dog_rates/status/889531135344209921

`doggo`, `pupper`, `floofer`, and `puppo` refers to the different dog stages. (via #WeRateDogs book on Amazon)
- `doggo`: a big pupper, usually older. May behave like a pupper, and is a pupper that has its life in order.
- `pupper`: a small doggo, usually younger. Can be equally, or even more mature than doggo. A doggo that is inexperienced and unprepared for responsibilities.
- `puppo`: transitional phase between a pupper and doggo. It is the equivalent of a teenager
- `floofer`: any dog really, but commonly used for dogs with excessive hair. 

In [88]:
df_dogs_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [87]:
df_dogs_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [86]:
df_dogs_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [95]:
# rating_numerator values
df_dogs_archive.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [124]:
pd.set_option('display.max_colwidth', -1)
df_dogs_archive[["text", "rating_numerator", "rating_denominator", "name"]]

,text,rating_numerator,rating_denominator,name
0,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13,10,Phineas
1,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",13,10,Tilly
2,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,12,10,Archie
3,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,13,10,Darla
4,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",12,10,Franklin
5,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,13,10,None
6,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,13,10,Jax
7,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,13,10,None
8,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,13,10,Zoey
9,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,14,10,Cassie


In [205]:
df_dogs_archive[df_dogs_archive.tweet_id == 866334964761202688.00]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
136,866334964761202691,nan,nan,2017-05-21 16:48:45 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Coco. At first I thought she was a cloud but clouds don't bork with such passion. 12/10 would hug softly https://t.co/W86h5dgR6c,nan,nan,NaN,"https://twitter.com/dog_rates/status/866334964761202691/photo/1,https://twitter.com/dog_rates/status/866334964761202691/photo/1",12,10,Coco,None,None,None,None


***Dataset 2: Image Prediction `df_image_pred`***

Column descriptions:
- `tweet_id`: last part of the tweet URL after "status/"
- `img_num`: image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images)
- `p1`: algorithm's #1 prediction for the image in the tweet
- `p1_conf`: how confident the algorithm is in its #1 prediction
- `p1_dog`: whether or not the #1 prediction is a breed of dog
- `p2` and `p3` equivalents are the algorithm's second and third most likely prediction, respectively

In [229]:
df_image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.47,True,collie,0.16,True,Shetland_sheepdog,0.06,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.51,True,miniature_pinscher,0.07,True,Rhodesian_ridgeback,0.07,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.60,True,malinois,0.14,True,bloodhound,0.12,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.41,True,redbone,0.36,True,miniature_pinscher,0.22,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.56,True,Rottweiler,0.24,True,Doberman,0.15,True


In [97]:
df_image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [98]:
df_image_pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


***Dataset 3: Tweets Information `df_tweets`***

In [227]:
df_tweets.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [102]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
tweet_id          2354 non-null int64
retweet_count     2354 non-null int64
favorite_count    2354 non-null int64
dtypes: int64(3)
memory usage: 55.2 KB


In [103]:
df_tweets.describe()

,tweet_id,retweet_count,favorite_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,3164.797366,8080.968564
std,6.852812e+16,5284.770364,11814.771334
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,624.500000,1415.000000
50%,7.194596e+17,1473.500000,3603.500000
75%,7.993058e+17,3652.000000,10122.250000
max,8.924206e+17,79515.000000,132810.000000


#### Tidiness Issues

- combine `df_tweets` DF with `df_dogs_archive` DF
- `retweeted_status_id` from `df_dogs_archive` DF are retweets from original tweets that will not be considered as they are duplicates

#### Quality issues

**combined `df_tweets` and `df_dogs_archive` DF**

- - Non-null `in_reply_to_status_id` and `in_reply_to_user_id` columns are replies to original tweets which won't be considered.
- Incorrect data formats (`timestamp`,  `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `doggo`, `floofer`, `pupper`, and `puppo`)
- `rating_numerator` and `rating_denominator` contain invalid ratings, i.e. numerator contain large range of values (0-1776) and denominator  not equal to 10, respectively. These ratings are not aligned with the tweet's text.
- `name` of dogs are automatically extracted after "This is", but some return inaccurate results, such as  "quite" or "a".
- `doggo`, `floofer`, `pupper`, and `puppo` columns contain many "None" values.
- Missing values in several columns (`in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, and `expanded_urls`)

<a id="cleaning"></a>
### Cleaning Data

First, create copies of the DataFrames.

In [253]:
df_dogs_archive_copy = df_dogs_archive.copy()
df_image_pred_copy = df_image_pred.copy()
df_tweets_copy = df_tweets.copy()

# Display full numbers
pd.set_option('display.float_format', '{:.2f}'.format)

#### Cleaning Tidiness Issues

- `retweet_status_id` from `df_dogs_archive` DF are retweets from original tweets that will not be considered as they are duplicates
- combining `df_tweets` DF with `df_dogs_archive` DF

##### Define, Code and Test
`retweeted_status_id` from `df_dogs_archive` DF are retweets from original tweets that will not be considered as they are duplicates. Those observations which do not have NaN values will be removed from DF.

In [254]:
# total observations
len(df_dogs_archive_copy)

2356

In [258]:
# non-null retweeted_status_id values  
sum(df_dogs_archive_copy.retweeted_status_id.notnull())

181

In [260]:
# tweet_id that have a retweeted_status_id
df_dogs_archive_copy = df_dogs_archive_copy[df_dogs_archive_copy.retweeted_status_id.isnull()]

In [261]:
df_dogs_archive_copy

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,nan,nan,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,nan,nan,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",nan,nan,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,nan,nan,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,nan,nan,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,nan,nan,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",nan,nan,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None
5,891087950875897856,nan,nan,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,nan,nan,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,None,None,None,None,None
6,890971913173991426,nan,nan,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,nan,nan,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13,10,Jax,None,None,None,None
7,890729181411237888,nan,nan,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,nan,nan,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13,10,None,None,None,None,None
8,890609185150312448,nan,nan,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,nan,nan,NaN,https://twitter.com/dog_rates/status/890609185150312448/photo/1,13,10,Zoey,None,None,None,None
9,89024025534

In [262]:
# Should return 2356-181=2175 observations
len(df_dogs_archive_copy)

2175

In [251]:
# Should return 0 
sum(df_dogs_archive_copy.retweeted_status_id.notnull())

0

##### Define, Code, Test
Combine `df_tweets` DF with `df_dogs_archive` DF to form a new DF called `df_dogs_tweets` using the merge function. 

In [263]:
# Merge two DF
df_dogs_tweets = df_dogs_archive_copy.merge(df_tweets, how="outer", left_on="tweet_id", 
                                       right_on="tweet_id")

In [264]:
df_dogs_tweets

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
0,892420643555336193,nan,nan,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,nan,nan,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13.00,10.00,Phineas,None,None,None,None,8853,39467
1,892177421306343426,nan,nan,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",nan,nan,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13.00,10.00,Tilly,None,None,None,None,6514,33819
2,891815181378084864,nan,nan,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,nan,nan,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12.00,10.00,Archie,None,None,None,None,4328,25461
3,891689557279858688,nan,nan,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,nan,nan,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13.00,10.00,Darla,None,None,None,None,8964,42908
4,891327558926688256,nan,nan,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",nan,nan,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12.00,10.00,Franklin,None,None,None,None,9774,41048
5,891087950875897856,nan,nan,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,nan,nan,NaN,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13.00,10.00,None,None,None,None,None,3261,20562
6,890971913173991426,nan,nan,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,nan,nan,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,https://twitter.com/dog_rates/status/890971913173991426/photo/1",13.00,10.00,Jax,None,None,None,None,2158,12041
7,890729181411237888,nan,nan,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,nan,nan,NaN,"https://twitter.com/dog_rates/status/890729181411237888/photo/1,https://twitter.com/dog_rates/status/890729181411237888/photo/1",13.00,10.00,None,None,None,None,None,16716,56848
8,890609185150312448,nan,nan,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boa

#### Quality issues

**`df_dogs_tweets` DF**

- Non-null `in_reply_to_status_id` and `in_reply_to_user_id` columns are replies to original tweets. These will be removed from the DF.
- Incorrect data formats: change `timestamp` to a datetime object, `doggo`, `floofer`, `pupper`, and `puppo` as Boolean. `retweeted_status_id`, `retweeted_status_user_id`, and `retweeted_status_timestamp` are no longer needed after cleaning tidiness issues above.
- `rating_numerator` contain invalid ratings, i.e. numerator contain large range of values (0-1776). Some ratings are not aligned with the tweet's text.
- `rating_denominator` contain invalid entries, where denominator is not equal to 10, respectively.
- `name` of dogs are automatically extracted after "This is", but some return inaccurate results, such as  "quite" or "a".
- `doggo`, `floofer`, `pupper`, and `puppo` columns contain many "None" values.

In [621]:
# Create copy of newly created DF
df_dogs_tweets_copy = df_dogs_tweets.copy()

##### Define, Code, Test

- Non-null `in_reply_to_status_id` and `in_reply_to_user_id` columns are replies to original tweets. These will be removed from the DF.

In [622]:
# Non-null values
df_dogs_tweets_copy[df_dogs_tweets_copy.in_reply_to_status_id.notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count
29,886267009285017600,886266357075128320.00,2281181600.00,2017-07-15 16:51:35+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@NonWhiteHat @MayhewMayhem omg hello tanner you are a scary good boy 12/10 would pet with extreme caution,nan,nan,NaN,NaN,12.00,10.00,None,None,None,None,None,4,117
52,881633300179243008,881607037314052096.00,47384430.00,2017-07-02 21:58:53+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s,nan,nan,NaN,NaN,17.00,10.00,None,None,None,None,None,7,129
61,879674319642796034,879553827334172672.00,3105440746.00,2017-06-27 12:14:36+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@RealKentMurphy 14/10 confirmed,nan,nan,NaN,NaN,14.00,10.00,None,None,None,None,None,10,315
101,870726314365509632,870726202742493184.00,16487760.00,2017-06-02 19:38:25+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is reserved for dogs,nan,nan,NaN,NaN,10.00,10.00,None,None,None,None,None,3,121
130,863427515083354112,863425645568774144.00,77596200.00,2017-05-13 16:15:35+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","@Jack_Septic_Eye I'd need a few more pics to polish a full analysis, but based on the good boy content above I'm leaning towards 12/10",nan,nan,NaN,NaN,12.00,10.00,None,None,None,None,None,105,2363
131,863079547188785154,667152164079423488.00,4196983835.00,2017-05-12 17:12:53+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Ladies and gentlemen... I found Pipsy. He may have changed his name to Pablo, but he never changed his love for the sea. Pupgraded to 14/10 https://t.co/lVU5GyNFen",nan,nan,NaN,https://twitter.com/dog_rates/status/863079547188785154/photo/1,14.00,10.00,None,None,None,None,None,1195,9094
156,857214891891077121,857156678055342080.00,180670967.00,2017-04-26 12:48:51+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@Marc_IRL pixelated af 12/10,nan,nan,NaN,NaN,12.00,10.00,None,None,None,None,None,20,242
159,856526610513747968,855818117272018944.00,4196983835.00,2017-04-24 15:13:52+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY HI AFTER ALL. PUPGRADED TO A 14/10. WOULD BE AN HONOR TO FLY WITH https://t.co/p1hBHCmWnA",nan,nan,NaN,https://twitter.com/dog_rates/status/856526610513747968/photo/1,14.00,10.00,None,None,None,None,None,2068,12446
160,856288084350160898,856286004109553664.00,279280991.00,2017-04-23 23:26:03+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@xianmcguire @Jenna_Marbles Kardashians wouldn't be famous if as a society we didn't place enormous value on what they do. The dogs are very deserving of their 14/10,nan,nan,NaN,NaN,14.00,10.00,None,None,None,None,None,17,545
162,855862651834028034,855861584463351808.00,194351775.00,2017-04-22 19:15:32+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",@dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research,nan,nan,NaN,NaN,420.00,10.00,None,None,None,None,None,27,320


In [623]:
# Total non-null columns for both columns
sum(df_dogs_tweets.in_reply_to_status_id.notnull() & df_dogs_tweets.in_reply_to_user_id.notnull())

78

In [624]:
# Total non-null columns for in_reply_to_status_id
sum(df_dogs_tweets.in_reply_to_status_id.notnull())

78

In [625]:
# Total non-null columns for in_reply_to_user_id
sum(df_dogs_tweets.in_reply_to_user_id.notnull())

78

In [626]:
# Remove all non-null values for columns in_reply_to_status_id 
# as they are also the same for in_reply_to_user_id 
df_dogs_tweets_copy = df_dogs_tweets_copy[df_dogs_tweets.in_reply_to_status_id.isnull()]

In [627]:
# Should return 0 as all non-null values are now removed
sum(df_dogs_tweets_copy.in_reply_to_status_id.notnull())

0

In [628]:
# Should return 0
sum(df_dogs_tweets_copy.in_reply_to_user_id.notnull())

0

##### Define, Code, Test

Change `timestamp` to a datetime object.

## <span style="color:red">TO DO</span> 
`doggo`, `floofer`, `pupper`, and `puppo` as Boolean, where None.

In [629]:
df_dogs_tweets_copy.timestamp = pd.to_datetime(df_dogs_tweets_copy.timestamp)

In [630]:
#df_dogs_tweets[["doggo", "floofer", "pupper", "puppo"]] = df_dogs_tweets[["doggo", "floofer", "pupper", "puppo"]].map(bool_dict)

In [631]:
df_dogs_tweets_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2276 entries, 0 to 2353
Data columns (total 19 columns):
tweet_id                      2276 non-null int64
in_reply_to_status_id         0 non-null float64
in_reply_to_user_id           0 non-null float64
timestamp                     2097 non-null datetime64[ns, UTC]
source                        2097 non-null object
text                          2097 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 2094 non-null object
rating_numerator              2097 non-null float64
rating_denominator            2097 non-null float64
name                          2097 non-null object
doggo                         2097 non-null object
floofer                       2097 non-null object
pupper                        2097 non-null object
puppo                         2097 non-null object
retweet_count          

##### Define, Test, Code

`rating_numerator` and `rating_denominator` contain invalid ratings, including incorrectly extracted dates instead of ratings or ratings which are not out of 10. Those with ratings from dates will be reextracted, and those with ratings that are not out of 10 will be converted to out of 10. 

In [632]:
ratings_tweak = df_dogs_tweets_copy[["text", "rating_numerator", "rating_denominator"]].query("rating_denominator != 10.00")
ratings_tweak[ratings_tweak.rating_numerator.notnull()]

,text,rating_numerator,rating_denominator
363,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.00,70.00
429,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24.00,7.00
733,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.00,150.00
890,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9.00,11.00
942,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.00,170.00
987,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4.00,20.00
1024,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50.00,50.00
1050,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.00,90.00
1075,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.00,80.00
1095,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.00,50.00


As there are few records, these can be completely quickly manually.

- The following index contain incorrect ratings: 
    - No valid ratings in tweet: 429 -> given null values
    - Incorrect extraction of ratings: 890, 987, 1024, 1483, 2154 -> change to correct values
- The remaining index require the denominators to be 10 (index: 363, 733, 942, 1050, 1075, 1095, 1172, 1254, 1456, 1600, 1664)

In [633]:
# Replace index 429 with null values
df_dogs_tweets_copy.at[429, "rating_numerator"] = np.nan
df_dogs_tweets_copy.at[429, "rating_denominator"] = np.nan

In [634]:
# Replace incorrect ratings with correct ones 
ratings_dict = {890: [14, 10], 987: [13, 10], 1024: [11, 10], 1483: [10, 10], 2154: [9, 10]}

for idx in ratings_dict.keys():
    df_dogs_tweets_copy.at[idx, "rating_numerator"] = ratings_dict[idx][0]
    df_dogs_tweets_copy.at[idx, "rating_denominator"] = ratings_dict[idx][1]

In [635]:
# Convert the remaining index to denominator 10, and correspondingly divide numerator by divisor
ratings_ls = [363, 733, 942, 1050, 1075, 1095, 1172, 1254, 1456, 1600, 1664]
for idx in ratings_ls:
    numerator = df_dogs_tweets_copy.at[idx, "rating_numerator"]
    denominator = df_dogs_tweets_copy.at[idx, "rating_denominator"]
    divisor = denominator//10
    df_dogs_tweets_copy.at[idx, "rating_numerator"] = numerator/10.00
    df_dogs_tweets_copy.at[idx, "rating_denominator"] = 10.00

In [636]:
df_dogs_tweets_copy["rating_numerator"].value_counts()

12.00      486
10.00      437
11.00      414
13.00      288
9.00       153
8.00       99 
7.00       51 
14.00      39 
5.00       34 
6.00       33 
3.00       19 
4.00       15 
2.00       9  
1.00       4  
0.00       1  
27.00      1  
9.90       1  
8.80       1  
26.00      1  
12.10      1  
8.40       1  
420.00     1  
20.40      1  
4.40       1  
16.50      1  
1776.00    1  
75.00      1  
14.40      1  
4.50       1  
Name: rating_numerator, dtype: int64

In [637]:
df_dogs_tweets_copy["rating_denominator"].value_counts()

10.00    2096
Name: rating_denominator, dtype: int64

#### Define, Code, Test
- `name` of dogs are automatically extracted after "This is", but some return inaccurate results, such as  "quite" or "a". Replace "None" with null values. 

In [638]:
df_dogs_tweets_copy.name = df_dogs_tweets_copy.name.replace("None", np.nan)

In [639]:
pd.set_option('display.max_rows', 1000)
df_dogs_tweets_copy[df_dogs_tweets_copy.name.str.islower() == True][["text", "name"]]

,text,name
21,I've yet to rate a Venezuelan Hover Wiener. This is such an honor. 14/10 paw-inspiring af (IG: roxy.thedoxy) https://t.co/20VrLAA8ba,such
53,Here is a pupper approaching maximum borkdrive. Zooming at never before seen speeds. 14/10 paw-inspiring af \n(IG: puffie_the_chow) https://t.co/ghXBIIeQZF,a
147,We only rate dogs. This is quite clearly a smol broken polar bear. We'd appreciate if you only send dogs. Thank you... 12/10 https://t.co/g2nSyGenG9,quite
167,"Guys, we only rate dogs. This is quite clearly a bulbasaur. Please only send dogs. Thank you... 12/10 human used pet, it's super effective https://t.co/Xc7uj1C64x",quite
282,There's going to be a dog terminal at JFK Airport. This is not a drill. 10/10 \nhttps://t.co/dp5h9bCwU7,not
310,"Occasionally, we're sent fantastic stories. This is one of them. 14/10 for Grace https://t.co/bZ4axuH6OK",one
450,We only rate dogs. Please stop sending in non-canines like this Freudian Poof Lion. This is incredibly frustrating... 11/10 https://t.co/IZidSrBvhi,incredibly
524,Here is a perfect example of someone who has their priorities in order. 13/10 for both owner and Forrest https://t.co/LRyMrU7Wfq,a
647,Guys this is getting so out of hand. We only rate dogs. This is a Galapagos Speed Panda. Pls only send dogs... 10/10 https://t.co/8lpAGaZRFn,a
662,We only rate dogs. Pls stop sending in non-canines like this Arctic Floof Kangaroo. This is very frustrating. 11/10 https://t.co/qlUDuPoE3d,very


The following dictionary shows the individual cases where the incorrect names are extracted. There is no obvious pattern to extract the correct names programmatically, so a manual approach is required. The remaining index's text contain no names, so the names will be changed to NaN values correspondingly. 

In [640]:
names_dict = {689: "Zoey", 817: "Quizno", 1776: "Kip", 1885: "Jacob", 1887: "Rufus", 
              1937: "Spork", 1946: "Cherokee", 1949: "Hemry", 1967: "Alphred", 1982: "Alfredo",
              2012: "Leroi", 2025: "Berta", 2039: "Chuk", 2056: "Alfonso", 2070: "Cheryl",
              2076: "Jessiga", 2083: "Klint", 2092: "Kohl", 2106: "Daryl", 2123: "Pepe",
              2130: "Octaviath", 2133: "Johm"}

In [641]:
# Iterate through dict and change the names according to the values
for idx in names_dict.keys():
    df_dogs_tweets_copy.at[idx, "name"] = names_dict[idx]

In [642]:
for idx in names_dict.keys():
    print(df_dogs_tweets_copy.name[idx])

Zoey
Quizno
Kip
Jacob
Rufus
Spork
Cherokee
Hemry
Alphred
Alfredo
Leroi
Berta
Chuk
Alfonso
Cheryl
Jessiga
Klint
Kohl
Daryl
Pepe
Octaviath
Johm


In [643]:
# All other tweets which contain no names are replaced by NaN values
names_idx = df_dogs_tweets_copy[df_dogs_tweets_copy.name.str.islower() == True].index

for idx in names_idx:
    df_dogs_tweets_copy.at[idx, "name"] = np.nan

In [644]:
# Should return no rows
df_dogs_tweets_copy[df_dogs_tweets_copy.name.str.islower() == True]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,retweet_count,favorite_count


In [647]:
df_dogs_tweets_copy.name.value_counts()

Charlie           11
Lucy              11
Cooper            10
Oliver            10
Tucker            9 
Penny             9 
Sadie             8 
Winston           8 
Lola              8 
Daisy             7 
Toby              7 
Bailey            6 
Jax               6 
Oscar             6 
Bo                6 
Bella             6 
Koda              6 
Stanley           6 
Dave              5 
Scout             5 
Chester           5 
Rusty             5 
Milo              5 
Louis             5 
Buddy             5 
Bentley           5 
Leo               5 
Boomer            4 
Scooter           4 
Reggie            4 
Clark             4 
Phil              4 
Derek             4 
Chip              4 
Alfie             4 
Oakley            4 
Brody             4 
Sammy             4 
Gus               4 
Bear              4 
Cassie            4 
Duke              4 
Finn              4 
Jack              4 
Dexter            4 
Zoey              4 
Archie            4 
Jerry        

##### Define, Code, Test

`doggo`, `floofer`, `pupper`, and `puppo` columns contain many "None" values.

In [648]:
df_dogs_tweets_copy[["text", "doggo", "floofer", "pupper", "puppo"]]

,text,doggo,floofer,pupper,puppo
0,This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,None,None,None,None
1,"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",None,None,None,None
2,This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,None,None,None,None
3,This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,None,None,None,None
4,"This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",None,None,None,None
5,Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,None,None,None,None
6,Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\nhttps://t.co/Zr4hWfAs1H https://t.co/tVJBRMnhxl,None,None,None,None
7,When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10 https://t.co/v0nONBcwxq,None,None,None,None
8,This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek https://t.co/9TwLuAGH0b,None,None,None,None
9,This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A,doggo,None,None,None


<a id="eda"></a>
## Exploratory Data Analysis